In [11]:
import pandas as pd
import re
import openai
import re
import requests
import pandas as pd
from openai.embeddings_utils import get_embedding, cosine_similarity
from transformers import GPT2TokenizerFast
import tiktoken

API_KEY = ""
RESOURCE_ENDPOINT = "" 

openai.api_type = "azure"
openai.api_key = API_KEY
openai.api_base = RESOURCE_ENDPOINT
openai.api_version = "2022-12-01"

url = openai.api_base + "/openai/deployments?api-version=2022-12-01"

r = requests.get(url, headers={"api-key": API_KEY})
print(r)



<Response [200]>


In [13]:
# s is input text
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    
    return s



## Data Loading

In [54]:
df = pd.read_parquet("final_df.parquet")
df.head()

,Consumer complaint narrative,Issue,n_tokens,embedding,prediction
1665,I know at this point I do n't know what can be...,Dealing with my lender or servicer,842,"[-0.01611567847430706, -0.024448800832033157, ...",Can't repay my loan
422,My Name is XXXX. While I attended school at th...,Can't repay my loan,713,"[-0.009162590838968754, -0.006951234769076109,...",Can't repay my loan
4771,I have had a Macy 's credit account for years ...,Delinquent account,622,"[0.0075323572382330894, 0.0036780808586627245,...",Unable to get credit report/credit score
1046,Incorporatess by reference all preceding parag...,"Application, originator, mortgage broker",808,"[0.004820031579583883, -0.010541003197431564, ...","Loan modification,collection,foreclosure"
14628,A buyer has purchased a watch from me online a...,Transaction issue,523,"[0.005065633915364742, -0.03737545758485794, 0...",Transaction issue


In [55]:
#apply text normalization
df['Consumer complaint narrative']= df["Consumer complaint narrative"].apply(lambda x : normalize_text(x))




In [56]:
#optional, use this to filter/cut long text (longer than 8192 tokens)
tokenizer = tiktoken.get_encoding("cl100k_base")
df['n_tokens'] = df["Consumer complaint narrative"].apply(lambda x: len(tokenizer.encode(x)))
df = df[df.n_tokens<8192]
len(df)

1000

In [17]:
df['embedding'] = df["Consumer complaint narrative"].apply(lambda x : get_embedding(x, engine = 'text-embedding-ada-002'))


#### Prepare the category description : embedding map

In [24]:
#get the category descriptions (unlike normal classification, the more detail, more clear, more distinctive description the better)
# produce the embedding for each category description 
import numpy as np
issues = np.unique(df.Issue)
issue_emb = {issue:get_embedding(issue, engine = 'text-embedding-ada-002')for issue in issues}

In [52]:
issue_emb['APR or interest rate']

[-0.011190177872776985,
 -0.018179483711719513,
 0.05631539225578308,
 0.0070075299590826035,
 -0.013513871468603611,
 -0.005822902079671621,
 0.0226901825517416,
 0.038345497101545334,
 0.023528534919023514,
 -0.013522983528673649,
 -0.027829645201563835,
 0.0005404865369200706,
 0.017969895154237747,
 0.007463156245648861,
 -0.00269502867013216,
 -0.0046063801273703575,
 0.005708995740860701,
 -0.017705632373690605,
 0.0010855293367058039,
 0.009586374275386333,
 -0.023419184610247612,
 -0.015017437748610973,
 -0.027756744995713234,
 0.024603812023997307,
 0.011682254262268543,
 0.00034342819708399475,
 -0.011472666636109352,
 0.005722664296627045,
 0.0008679677848704159,
 0.011199290864169598,
 0.013905709609389305,
 -0.022671956568956375,
 0.008319733664393425,
 0.026772592216730118,
 -0.013604996725916862,
 0.018316170200705528,
 -0.015846677124500275,
 0.019774174317717552,
 -0.0024262091610580683,
 -0.033461183309555054,
 0.03162045404314995,
 0.004212263505905867,
 -0.027045967

In [41]:
import numpy as np  
  
import numpy as np  
  
def categorize(input_vector):  
    """  
    Given an input vector and a dictionary of label vectors,  
    returns the label with the highest cosine similarity to the input vector.  
    """  
    max_similarity = float('-inf')  
    max_label = None  
      
    # Compute cosine similarity between input vector and each label vector  
    for label, vector in issue_emb.items():  
        cosine_similarity = np.dot(input_vector, vector) / (np.linalg.norm(input_vector) * np.linalg.norm(vector))  
          
        # Update max_similarity and max_label if necessary  
        if cosine_similarity > max_similarity:  
            max_similarity = cosine_similarity  
            max_label = label  
      
    return max_label  


### Prediction or matching

In [42]:
df["prediction"] = df["embedding"].apply(categorize)

In [45]:
df[df["prediction"] ==df["Issue"]].count()/df.shape[0]

Consumer complaint narrative    0.202
Issue                           0.202
n_tokens                        0.202
embedding                       0.202
prediction                      0.202
dtype: float64

In [49]:
df[['Consumer complaint narrative','Issue','prediction']].head()

,Consumer complaint narrative,Issue,prediction
1665,I know at this point I do n't know what can be...,Dealing with my lender or servicer,Can't repay my loan
422,My Name is XXXX. While I attended school at th...,Can't repay my loan,Can't repay my loan
4771,I have had a Macy 's credit account for years ...,Delinquent account,Unable to get credit report/credit score
1046,Incorporatess by reference all preceding parag...,"Application, originator, mortgage broker","Loan modification,collection,foreclosure"
14628,A buyer has purchased a watch from me online a...,Transaction issue,Transaction issue
